In [10]:
import numpy as np
import scipy.linalg as la
import time

# Problem 12.11

The Algorithm:

$S_{t+1} = S_t + x_{t+1}x_{t+1}^T $

$\gamma_{t+1} = S_{t+1}^{-1}x_{t+1} $

$e_{t+1} = y_{t+1} - x_{t+1}^T\hat{\beta}_t $

$\hat{\beta}_{t+1} = \hat{\beta}_t + \gamma_{t+1}e_{t+1} $

First, note that showing that $\hat{\beta}_{t+1} = S_{t+1}^{-1}X_{t+1}^TY_{t+1}$ is equivilant to showing $S_{t+1}\hat{\beta}_{t+1} = X_{t+1}^TY_{t+1}$. Now observe that $ X_{t+1}^TY_{t+1} = X_t^TY_t + x_{t+1}y_{t+1}$. Below we calculate using the proposed algorithm definitions above.

$$
S_{t+1}\hat{\beta}_{t+1} 
= S_{t+1}(\hat{\beta}_t + \gamma_{t+1}e_{t+1}) 
= S_{t+1}(\hat{\beta}_t + (S_{t+1}^{-1}x_{t+1})(y_{t+1} - x_{t+1}^T\hat{\beta}_t)) 
= S_{t+1}\hat{\beta}_t + x_{t+1}(y_{t+1} - x_{t+1}^T\hat{\beta}_t) 
$$

$$
= (S_t + x_{t+1}x_{t+1}^T)(S_t^{-1}X_t^TY_t) + x_{t+1}(y_{t+1} - x_{t+1}^T\hat{\beta}_t) 
= X_t^TY_t + x_{t+1}x_{t+1}^TS_t^{-1}X_t^TY_t + x_{t+1}y_{t+1} - x_{t+1}x_{t+1}^T\hat{\beta}_t
$$

$$
= X_t^TY_t + x_{t+1}y_{t+1} 
= X_{t+1}^TY_{t+1}
$$

And thus we see the algorithm works.

# Problem 12.12

The Sherman-Morrison-Woodbury formula is the following: 
$(A + uv^T)^{-1} = A^{-1} - \frac{A^{-1}uv^TA^{-1}}{1 + v^TA^{-1}u}$

Now observe the following, using a straightforward application of the Sherman-Morrison-Woodbury:

$$
S_{t+1}^{-1} 
= (S_t + x_{t-1}x_{t+1}^T)^{-1} 
= S_t^{-1} - \frac{S_t^{-1}x_{t+1}x_{t+1}^TS_t^{-1}}{1 + x_{t+1}^TS_t^{-1}x_{t+1}}
$$

# Problem 12.13

Use the data from https://archive.ics.uci.edu/ml/datasets/BlogFeedback.

In [11]:
train = np.loadtxt('BlogFeedback/blogData_train.csv', 
                   delimiter=',')
Xtrain = train[:,:-1]
ytrain = train[:,-1]

In [12]:
class RLS(object):
    
    def __init__(self, X, Y, PCA=5):
        '''
        if PCA == -1 dont do PCA
        ''' 
        n = X.shape[0]
        if PCA != -1 and PCA < X.shape[1]:
            self.PCA = True
            self.Xmean = X.mean(axis=0)
            X = X - self.Xmean
            self.projection = la.svd(X, full_matrices=False)[-1][:PCA].T
            X = X @ self.projection # reduce X down to PCA dimensions
        else:
            self.PCA = False
        X = np.concatenate((np.ones(n).reshape((n,1)), X), axis=1)
        #self.S = X.T @ X
        self.S_inv = la.inv(X.T @ X)
        self.beta = self.S_inv @ X.T @ Y
    
    def update(self,X,Y):
        if self.PCA:
            X = (X - self.Xmean) @ self.projection
        X = np.concatenate(([1],X))
        #self.S = self.S + np.outer(X,X)
        Sx = self.S_inv @ X
        self.S_inv = self.S_inv - (np.outer(Sx,Sx))/(1 + X @ Sx)
        gamma = self.S_inv @ X
        e = Y - X @ self.beta
        self.beta = self.beta + gamma * e

In [13]:
rls = RLS(Xtrain, ytrain)

In [ ]:
newdata = np.loadtxt('BlogFeedback/blogData_test-2012.02.01.00_00.csv', 
                     delimiter=',')
Xnew = newdata[0,:-1]
ynew = newdata[0,-1]

In [ ]:
start = time.clock()
rls.update(Xnew, ynew)
update_time = time.clock() - start

In [22]:
start = time.clock()
RLS(np.concatenate((Xtrain,Xnew.reshape(1,Xnew.shape[0]))), 
    np.concatenate((ytrain, [ynew])))
ols_time = time.clock() - start

In [24]:
print("Update Time:\t{}\nOLS Time:\t{}".format(update_time, ols_time))

Update Time:	0.0002293335215044279
OLS Time:	1.9227170647934884


The complexity of the OLS solution is mega huge in comparison to the update, and it only gets bigger because n increases with each step. The update algorithm is primarily parameterized by the size of the feature space, which stays constant (here, we've chosen to project down to a 5-dimensional space plus a constant, so it's only 6 dimensional). However, the OLS fitting is parameterized by both the feature space size and the number of observations, so as time goes on the observations grow and the matrix multiplication costs increase, but for update the cost stays constant. As far as I can tell the order of the complexity is about the same for both (somewhere around square or cubic), especially if you use an alternate method instead of inverting $S_t$ in the OLS fit.